# Generate Performance Plot

## Import Packages

In [1]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import NumeralTickFormatter, Label
from bokeh.palettes import Viridis8
from bokeh.io import export_png

output_notebook()

Loading BokehJS ...

## Retrieve and Process Data

In [2]:
df = pd.read_html('https://github.com/TDAmeritrade/stumpy', match='STUMPED.256')[0]
df = df.rename(columns={'n = 2i': 'n'})
df['GPU-STOMP'] = pd.to_timedelta(df['GPU-STOMP'])
df['STUMP.16'] = pd.to_timedelta(df['STUMP.16'])
df['STUMPED.128'] = pd.to_timedelta(df['STUMPED.128'])
df['STUMPED.256'] = pd.to_timedelta(df['STUMPED.256'])
df['GPU-STUMP.1'] = pd.to_timedelta(df['GPU-STUMP.1'])
df['GPU-STUMP.2'] = pd.to_timedelta(df['GPU-STUMP.2'])
df['GPU-STUMP.DGX1'] = pd.to_timedelta(df['GPU-STUMP.DGX1'])
df['GPU-STUMP.DGX2'] = pd.to_timedelta(df['GPU-STUMP.DGX2'])

df.head()

,i,n,GPU-STOMP,STUMP.16,STUMPED.128,STUMPED.256,GPU-STUMP.1,GPU-STUMP.2,GPU-STUMP.DGX1,GPU-STUMP.DGX2
0,6.0,64,00:00:10,00:00:00,00:00:05.770000,00:00:06.080000,00:00:00.010000,00:00:01.630000,00:00:07.950000,NaT
1,7.0,128,00:00:10,00:00:00,00:00:05.930000,00:00:07.290000,00:00:00.020000,00:00:01.640000,00:00:06.880000,00:00:30.980000
2,8.0,256,00:00:10,00:00:00.010000,00:00:05.950000,00:00:07.590000,00:00:00.040000,00:00:01.640000,00:00:06.480000,00:00:27.270000
3,9.0,512,00:00:10,00:00:00.020000,00:00:05.970000,00:00:07.470000,00:00:00.090000,00:00:01.650000,00:00:06.370000,00:00:29.240000
4,10.0,1024,00:00:10,00:00:00.040000,00:00:05.690000,00:00:07.640000,00:00:00.190000,00:00:01.740000,00:00:06.330000,00:00:29.580000


In [3]:
dfs = {
    'GPU-STOMP': df[['n', 'GPU-STOMP']],
    'STUMP.16': df[['n', 'STUMP.16']],
    'STUMPED.128': df[['n', 'STUMPED.128']],
    'STUMPED.256': df[['n', 'STUMPED.256']],
    'GPU-STUMP.1': df[['n', 'GPU-STUMP.1']],
    'GPU-STUMP.2': df[['n', 'GPU-STUMP.2']],
    'GPU-STUMP.DGX1': df[['n', 'GPU-STUMP.DGX1']],
    'GPU-STUMP.DGX2': df[['n', 'GPU-STUMP.DGX2']],
}

line_dashes = {
    'GPU-STOMP': '10 10',
    'STUMP.16': '5 5',
    'STUMPED.128': 'solid',
    'STUMPED.256': '15 15',
    'GPU-STUMP.1': 'solid',
    'GPU-STUMP.2': 'solid',
    'GPU-STUMP.DGX1': 'solid',
    'GPU-STUMP.DGX2': 'solid',
     #‘^(\d+(\s+\d+)*)?$’
}

for k in dfs.keys():
    dfs[k] = dfs[k].dropna()

## Plot Performance Results

In [4]:
p = figure(plot_width=1000)

for i, k in enumerate(dfs.keys()):
    p.line(dfs[k].iloc[:, 0], 
           dfs[k].iloc[:, 1]/pd.Timedelta('1 days'), 
           line_color=Viridis8[i], 
           line_width=4,
           line_dash=line_dashes[k],
           legend_label=k,
          )
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.xaxis.axis_label = 'Time Series Length (n)'
    p.yaxis.axis_label = 'Days'
    p.xaxis.axis_label_text_font_size = "16pt"
    p.yaxis.axis_label_text_font_size = "16pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.legend.location = "top_left"
    p.legend.label_text_font_size = "12pt"
    p.legend.glyph_width = 75

label = Label(x=80000000, y=5, text='Lower is "Better"', render_mode='css', text_font_size='16pt')
p.add_layout(label)

show(p)

In [5]:
export_png(p, 'performance.png')

'/Users/law978/Git/stumpy.git/docs/images/performance.png'